■環境構築方法

1.GoogleドライブMyDrive内にcut_movieフォルダを作成

2.cut_movieフォルダ内にinputフォルダ・outputフォルダを作成

3.inputフォルダ内に対象の動画をアップロード

4.コードを実行

■確認事項

・ランライムのタイプがGPUになっていることを確認

・GPUの使用確認時にh264_nvencがあることを確認


In [ ]:
#ffmpegのインストール
!git clone https://github.com/rokibulislaam/colab-ffmpeg-cuda.git
!cp -r ./colab-ffmpeg-cuda/bin/. /usr/bin/

#バージョン確認
!ffmpeg -version
#GPU使用可能状態かの確認
!ffmpeg -encoders | grep nvenc

#マウント
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import subprocess
import os
import glob

mount_dir="./drive/MyDrive/cut_movie/"

input_dir=os.path.join(mount_dir,"input")
output_dir=os.path.join(mount_dir,"output")

def get_movie(wk_dir):
    files = glob.glob(os.path.join(wk_dir,"*"))
    files = [x for x in files if x[-4:] == ".MOV" or x[-4:] == ".mp4"]
    return files


def cut_silent(movie, dB):
    output = subprocess.run(["ffmpeg", "-i",  movie,"-vcodec", "h264_nvenc", "-af", "silencedetect=noise={}dB:d=0.3".format(dB), "-f", "null", "-"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    ss = str(output)
    lines = ss.replace("\\r", "")
    lines = lines.split('\\n')
    time_list = []
    for line in lines:
        if "silencedetect" in line:
            words = line.split(" ")
            for i in range(len(words)):
                if "silence_start" in words[i]:
                    time_list.append(float(words[i+1]))
                if "silence_end" in words[i]:
                    time_list.append(float(words[i + 1]))
    silence_section_list = list(zip(*[iter(time_list)]*2))
    movie_name = os.path.basename(movie).split(".")
    if str(silence_section_list[0][0]) != "0.0":
        split_file1 = os.path.join(output_dir, movie_name[0] + "_0" + ".mp4")
        subprocess.run(["ffmpeg", "-ss", str(0), "-i", movie,"-vcodec", "h264_nvenc", "-t", str(silence_section_list[1][0]), split_file1], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for i in range(len(silence_section_list) - 1):
        split_file = os.path.join(output_dir, movie_name[0] + "_" + str(i+1) + ".mp4")
        subprocess.run(["ffmpeg", "-ss", str(silence_section_list[i][1]), "-i", movie,"-vcodec", "h264_nvenc", "-t", str(silence_section_list[i+1][0]-silence_section_list[i][1]), split_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(movie_name[0] + "_" + str(i+1) + ".mp4")

if __name__ == "__main__":
    
    movie_list = get_movie(input_dir)

    for movie in movie_list:
        print("処理する動画")
        print(movie)
        dB = input("カットする音量の閾値を入力(dB)、デフォルトの場合はそのままエンター ※デフォルト-33dB>", )
        if dB == "":
            dB = "-33"
        cut_silent(movie, dB)
    
    
    movie_list = get_movie(output_dir)
    for movie in movie_list:
        cut_file = subprocess.run(["ffmpeg", "-i", movie,"-vcodec", "h264_nvenc", "-f", "null", "-"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        s = str(cut_file)
        lines1 = s.replace("\\r", "")
        lines1 = lines1.split('\\n')
        for line in lines1:
            if "Duration" in line:
                words = line.split(" ")
                word = words[3].split(":")[2]
                word = word.replace(",", "")
                if float(word) < 0.5:
                    os.remove(movie)